# Initializing

In [1]:
#Active environment should be aws_env
!conda info | grep 'active env'

     active environment : aws_env
    active env location : /home/hassan101/anaconda3/envs/aws_env


In [2]:
#Get AWS credentials from environment
import os
import boto3
aws_akid = os.environ['AWS_KID']
aws_sak = os.environ['AWS_AK']

client = boto3.client('iam', aws_access_key_id=aws_akid, aws_secret_access_key= aws_sak)
users = client.list_users()
for key in users['Users']:
    print('Active User:', key['UserName'])

#Importing libraries
import pandas as pd
import io
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Active User: usr_hassan


# Getting test data locally

In [3]:
Xy_test = pd.read_csv('Xy_test.csv', index_col=0)
X_test = Xy_test.iloc[:,:-1]
y_test = Xy_test.iloc[:,[-1]]

print(X_test.shape)
print(y_test.shape)

(2113, 20)
(2113, 1)


In [4]:
#Dropping column not included in model
X_test.drop(columns='customerid', inplace=True)
print(X_test.shape)

(2113, 19)


# DEPLOYMENT AND MAKING PREDICTIONS (single sample only)

# Prepping sample

Taking random sample

In [5]:
import random

random_number = random.randint(0, len(y_test))

sample = X_test.iloc[[random_number]]
sample_label = y_test.iloc[random_number]['churn']

print('Actual label for data:', sample_label)
print(f'These are the features with data shape:{sample.shape}')

sample.to_csv('sample.csv', index=True)
sample.head()

Actual label for data: 0
These are the features with data shape:(1, 19)


,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
3153,female,no,no,no,23,yes,yes,dsl,yes,no,no,no,no,no,month-to-month,no,electronic_check,54.4,1249.25


In [6]:
#Loading model and params
import pickle
model, scaler, dv = pickle.load( open('model_stack_C=1.0.pkl', 'rb') )
display(model)
display(scaler)
display(dv)

LogisticRegression()

StandardScaler()

DictVectorizer(sparse=False)

Prepping sample as dictionary

In [7]:
sample_dic = sample.to_dict(orient='records') #This will save each row as dic in a list, and ignore index
sample_dic[0]

{'gender': 'female',
 'seniorcitizen': 'no',
 'partner': 'no',
 'dependents': 'no',
 'tenure': 23,
 'phoneservice': 'yes',
 'multiplelines': 'yes',
 'internetservice': 'dsl',
 'onlinesecurity': 'yes',
 'onlinebackup': 'no',
 'deviceprotection': 'no',
 'techsupport': 'no',
 'streamingtv': 'no',
 'streamingmovies': 'no',
 'contract': 'month-to-month',
 'paperlessbilling': 'no',
 'paymentmethod': 'electronic_check',
 'monthlycharges': 54.4,
 'totalcharges': 1249.25}

A quick check on predictions locally (we will convert to JSON anyway for local testing)

In [8]:
import json
import numpy as np

#Converting dictionary to JSON
data_as_json=json.dumps(sample_dic)
print('Data type:', type(data_as_json))

Data type: <class 'str'>


In [9]:
#Loading JSON back to dictionary
data_as_dic = json.loads(data_as_json)
print('Data type:', type(data_as_dic))
print('Data shape:', len(data_as_dic[0]))

#Making predictions from data dictionary
#First, applying OHE to dic, which will give array
Xohe_as_array = dv.transform(data_as_dic)
print('Data type:', type(Xohe_as_array))
print('Data shape:', Xohe_as_array.shape)
#Next, making predictions on scaled inputs
output = model.predict_proba(scaler.transform(Xohe_as_array))[:,1]
print('Predicted value:', '%.2f' % (output[0]))
print('Actual value:', sample_label)

Data type: <class 'list'>
Data shape: 19
Data type: <class 'numpy.ndarray'>
Data shape: (1, 46)
Predicted value: 0.22
Actual value: 0


# Predictions from Flask

Note that Flask is not recommended to run in prod environment. It won't handle multiple requests. Gunicorn/docker/cloud are better alternatives.

For flask, converting to JSON isn't needed. Requests are sent as dictionary.

Write a script to execute in deployment environment

In [10]:
%%writefile app_api.py

from flask import Flask, request,jsonify
import pickle
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
import numpy as np

app = Flask('customerchurn')
model, scaler, dv = pickle.load( open('model_stack_C=1.0.pkl', 'rb') )

@app.route('/predict_api', methods=['POST'])
def predict_api():
    data_as_dic=request.get_json()
    print('Data type:', type(data_as_dic))
    print('Data shape:', len(data_as_dic[0]))
    Xohe_as_array = dv.transform(data_as_dic)
    print('Data type:', type(Xohe_as_array))
    print('Data shape:', Xohe_as_array.shape)
    output = model.predict_proba(scaler.transform(Xohe_as_array))[:,1]
    print('Predicted value:', '%.2f' % (output[0]))
    return jsonify(output[0])

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=9696)

Overwriting app_api.py


Execute the above file in deployment environment as follows:

`
python app_api.py
`


Then make predictions like below:

In [41]:
import requests
url= 'http://localhost:9696/predict_api'
output=requests.post(url, json=sample_dic)
print('Predicted label:', output.json())

Predicted label: 0.22476433949190344


# Predictions on web

In [66]:
%%writefile app_web.py

from flask import Flask, request, app, jsonify, url_for, render_template
import pickle
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
import numpy as np

app = Flask('customerchurn')
model, scaler, dv = pickle.load( open('model_stack_C=1.0.pkl', 'rb') )

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/predict_web', methods=['POST'])
def predict():

    keys = [x for x in request.form.keys()]
    values = [x for x in request.form.values()]
    
    #Some numbers are converted to str, so we need to fix that.
    for i in range(0,len(values)):
        try:
            values[i] = float(values[i])
            print('changed')
        except ValueError:
            values[i] = str(values[i])
            print(values[i])

    data_as_dic = [dict(zip(keys, values))] #Our original sample is list of dictionaries so I used []
    print('Data type:', type(data_as_dic))
    print('Data shape:', len(data_as_dic[0]))
    Xohe_as_array = dv.transform(data_as_dic)
    print('Data type:', type(Xohe_as_array))
    print('Data shape:', Xohe_as_array.shape)
    output = model.predict_proba(scaler.transform(Xohe_as_array))[:,1]
    print('Predicted value:', '%.2f' % (output[0]))
    return render_template('home.html', prediction_text= "The churn probability is {}".format(output[0]))

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=9696)

Overwriting app_web.py


# Predictions on gunicorn

Run this code in the deployment environment to start the service:

```
gunicorn --bind=0.0.0.0:9696 app_api:app
```

Next you can send POST request as usual to get the predictions

In [67]:
import requests
url= 'http://localhost:9696/predict_api'
output=requests.post(url, json=sample_dic)
print('Predicted label:', output.json())

Predicted label: 0.22476433949190344


# Predictions on docker

Make an image with dockerfile instructions:

`
docker build -t customerchurn .
`

 then run this command in terminal (not in docker terminal)
```
docker run -it --rm -p 9696:9696 customerchurn
```
You need to make sure not to run this in docker terminal with VSC, since the port will be used up by VSC. Don't forget to map docker port with local by 9696:9696 mapping. The --rm removed all temporary files after container is closed.

In [68]:
#Once container is started, use the POST request to get predictions
import requests
url= 'http://localhost:9696/predict_api'
output=requests.post(url, json=sample_dic)
print('Predicted label:', output.json())

Predicted label: 0.22476433949190344


# Predictions on AWS Elastic beanstalk

First, you need to install the elastic beanstalk library in working environment

In [ ]:
# !pip install awsebcli

Check if installation is working fine:

In [77]:
!eb --version

EB CLI 3.20.3 (Python 3.8.0)


Initialize docker-based project:

In [78]:
!eb init -p docker customer-churn-service --profile usr_hassan

This profile usr_hassan contains AWS credentials stored in  ~/.aws/config file under [profile usr_hassan]

Check config.yml file for easticbeanstalk and make sure default_region is set to ap-southeast-2

In [79]:
!cat .elasticbeanstalk/config.yml

branch-defaults:
  main:
    environment: null
global:
  application_name: customer-churn-service
  branch: null
  default_ec2_keyname: null
  default_platform: Docker
  default_region: ap-southeast-2
  include_git_submodules: true
  instance_profile: null
  platform_name: null
  platform_version: null
  profile: usr_hassan
  repository: null
  sc: git
  workspace_type: Application


Test by executing locally (make sure Docker is operational)

`
eb local run --port 9696
`

Check if predictions are working locally:

In [80]:
import requests
url= 'http://localhost:9696/predict_api'
output=requests.post(url, json=sample_dic)
print('Predicted label:', output.json())

Predicted label: 0.22476433949190344


Now, create the service on cloud (this is not free):

`
eb create customer-churn-service
`


Once finished, copy URL from this line:


2023-02-14 00:47:19    INFO    Application available at life-exp-service.eba-snms8sq2.ap-southeast-2.elasticbeanstalk.com.

Modify URL section between {http://} and {/predict_api} for POST request and make predictions:

In [ ]:
import requests
url= 'http://life-exp-service.eba-snms8sq2.ap-southeast-2.elasticbeanstalk.com/predict_api'
output=requests.post(url, json=sample_dic)
print('Predicted label:', output.json())

Terminate the service from cloud afterwards to avoid paying a hefty bill